In [5]:
import sagemaker
import boto3
import pandas as pd

sagemaker_session=sagemaker.Session()

role=sagemaker.get_execution_role()

bucket=sagemaker_session.default_bucket()
prefix="nlp-model-demo"

C:\Users\anant\anaconda3\Lib\site-packages\pydantic\_internal\_fields.py:201: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\anant\AppData\Local\sagemaker\sagemaker\config.yaml


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 import boto3                                                                                │
│    3 import pandas as pd                                                                         │
│    4                                                                                             │
│ ❱  5 sagemaker_session=sagemaker.Session()                                                       │
│    6                                                                                             │
│    7 role=sagemaker.get_execution_role()                                                         │
│    8                                                                                             │
│                                                                                                  │
│ C:\Users\anant\anaconda3\Lib\site-packages\sagemaker\session.py:265 in __init__                  │
│                                                                                                  │
│    262 │   │   self.lambda_client = None                                                         │
│    263 │   │   self.settings = settings if settings else SessionSettings()                       │
│    264 │   │                                                                                     │
│ ❱  265 │   │   self._initialize(                                                                 │
│    266 │   │   │   boto_session=boto_session,                                                    │
│    267 │   │   │   sagemaker_client=sagemaker_client,                                            │
│    268 │   │   │   sagemaker_runtime_client=sagemaker_runtime_client,                            │
│                                                                                                  │
│ C:\Users\anant\anaconda3\Lib\site-packages\sagemaker\session.py:293 in _initialize               │
│                                                                                                  │
│    290 │   │                                                                                     │
│    291 │   │   self._region_name = self.boto_session.region_name                                 │
│    292 │   │   if self._region_name is None:                                                     │
│ ❱  293 │   │   │   raise ValueError(                                                             │
│    294 │   │   │   │   "Must setup local AWS configuration with a region supported by SageMaker  │
│    295 │   │   │   )                                                                             │
│    296                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Must setup local AWS configuration with a region supported by SageMaker.

In [3]:
pip install sagemaker

  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 17.7 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144577 sha256=1e085811987d6e895126c7cbc11426f40ba72b49748ab54fd7afe912217bc862
  Stored in directory: c:\users\anant\appdata\local\pip\cache\wheels\1f\be\48\13754633f1d08d1fbfc60d5e80ae1e5d7329500477685286cd
Successfully built antlr4-python3-runtime
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.10.0
    Uninstalling platformdirs-3.10.0:
      Successfully uninstalled platformdirs-3.10.0
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
   

In [9]:
df = pd.read_csv("Reviews.csv")  # Ensure the dataset is available locally

# Keep only required columns

df = df[["Text", "Score"]].dropna()

# Convert scores into binary sentiment (1 = Positive, 0 = Negative)

df["Sentiment"] = df["Score"].apply(lambda x: 1 if x > 3 else 0)

df = df[["Text", "Sentiment"]]

# Save processed data

df.to_csv("processed_reviews.csv", index=False)

# Upload to S3

s3 = boto3.client("s3")

s3.upload_file("processed_reviews.csv", bucket, f"{prefix}/processed_reviews.csv")

# Store S3 path

s3_train_data = f"s3://{bucket}/{prefix}/processed_reviews.csv"

print("Data uploaded to:", s3_train_data)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:21                                                                                   │
│                                                                                                  │
│   18                                                                                             │
│   19 s3 = boto3.client("s3")                                                                     │
│   20                                                                                             │
│ ❱ 21 s3.upload_file("processed_reviews.csv", bucket, f"{prefix}/processed_reviews.csv")          │
│   22                                                                                             │
│   23 # Store S3 path                                                                             │
│   24                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'bucket' is not defined

In [35]:
%%writefile train.py
import argparse
import os
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

Overwriting train.py


In [37]:
def train():

    # Argument parser for SageMaker input

    parser = argparse.ArgumentParser()

    parser.add_argument("--train_data", type=str, default=os.environ["SM_CHANNEL_TRAIN"])

    args = parser.parse_args()

 

    # Load dataset from the provided path

    train_data_path = os.path.join(args.train_data, "processed_reviews.csv")

    df = pd.read_csv(train_data_path)

 

    # Split data

    X = df["Text"]
    y=df["Sentiment"]

In [39]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words="english")),
    ("clf", LogisticRegression())
])
pipeline.fit(X, y)

    # Save trained model
model_path = os.path.join("/opt/ml/model", "model.joblib")
oblib.dump(pipeline, model_path)
print("Model saved at", model_path)
if __name__=="__main__":
    train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 pipeline = Pipeline([                                                                       │
│    2 │   ("tfidf", TfidfVectorizer(stop_words="english")),                                       │
│    3 │   ("clf", LogisticRegression())                                                           │
│    4 ])                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'Pipeline' is not defined